In [1]:
# uncomment this if you are missing some packages. Also you can use requirements.txt

# import sys
# !{sys.executable} -m pip install numpy scikit-learn pandas torch dash plotly ucimlrepo ipywidgets

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
from torch.utils.data import TensorDataset

from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.model_selection import ParameterGrid

# Получаем dataset
from ucimlrepo import fetch_ucirepo   
servo = fetch_ucirepo(id=87) 

## Выборка: [*Servo Data Set*](https://archive.ics.uci.edu/ml/datasets/Servo)

Согласно описанию, нужно предсказать rise time сервомеханизма, на основании 4-ых признаков, 2 из них это gain setting, остальные 2 тип механической связи (mechanical linkage). 
Признаки и цель
   1. motor: A,B,C,D,E (categorial)
   2. screw: A,B,C,D,E (categorial)
   3. pgain: 3,4,5,6 (integer)
   4. vgain: 1,2,3,4,5 (integer)
   5. class: 0.13 to 7.10 (real)

In [2]:
features = servo.data.features 
targets = servo.data.targets

# df = features + target
df = features
df['target'] = targets['class']
target = ['target']


Так как _motor_ и _screw_ -- категориальные признаки, конвертируем их в численные. Можем воспользоваться OneHotEncoder-ом из sklearn-a.

In [3]:
from sklearn.preprocessing import OneHotEncoder

categorical_columns = ['motor', 'screw']

#Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[categorical_columns])

#Create a DataFrame with the one-hot encoded columns
#We use get_feature_names_out() to get the column names for the encoded data
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the one-hot encoded dataframe with the original dataframe
df_encoded = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical columns
df_encoded = df_encoded.drop(categorical_columns, axis=1)
print(df_encoded)

     pgain  vgain    target  motor_A  motor_B  motor_C  motor_D  motor_E  \
0        5      4  0.281251      0.0      0.0      0.0      0.0      1.0   
1        6      5  0.506252      0.0      1.0      0.0      0.0      0.0   
2        4      3  0.356251      0.0      0.0      0.0      1.0      0.0   
3        3      2  5.500033      0.0      1.0      0.0      0.0      0.0   
4        6      5  0.356251      0.0      0.0      0.0      1.0      0.0   
..     ...    ...       ...      ...      ...      ...      ...      ...   
162      3      2  4.499986      0.0      1.0      0.0      0.0      0.0   
163      3      1  3.699967      0.0      1.0      0.0      0.0      0.0   
164      4      3  0.956256      0.0      0.0      1.0      0.0      0.0   
165      3      2  4.499986      1.0      0.0      0.0      0.0      0.0   
166      6      5  0.806255      1.0      0.0      0.0      0.0      0.0   

     screw_A  screw_B  screw_C  screw_D  screw_E  
0        0.0      0.0      0.0      

## Визуализация выборки

In [4]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from plotly.subplots import make_subplots


def extractMotor(motor_id):
    motor_id_df = df.loc[df['motor'] == motor_id] 
    return motor_id_df.drop(['motor'], axis=1)

app = Dash(__name__)

def setLayouts():
    layout_list = []
    for motor_id in ['A', 'B', 'C', 'D', 'E']:
        layout_list.append(html.Div([
        html.H4('motor ' + motor_id),
        dcc.Graph(id="graph" + motor_id),
        html.P("Target:"),
        dcc.RangeSlider(
            id='range-slider' + motor_id,
            min=0.13, max=7.10, step=0.25,
            marks={0.13: '0.13', 7.10: '7.10'},
            value=[0.13, 7.10]
        ),
        ]))
    return html.Div(layout_list)

# Немного криво, но прописывать 5 callback-oв ещё хуже
def genCallback(motor_id):
    callback_gen =  '''
@app.callback(
    Output("graph__MOTOR_ID__", "figure"),
    Input("range-slider__MOTOR_ID__", "value"))
def updateBarChart__MOTOR_ID__(slider_range):
    low, high = slider_range
    mask = (df.target > low) & (df.target < high)
    fig = px.scatter_3d(extractMotor('__MOTOR_ID__').loc[mask], 
                        x='pgain', y='vgain', z='target', color='screw',
                        color_discrete_map={
                            "A": "red",
                            "B": "green",
                            "C": "blue",
                            "D": "goldenrod",
                            "E": "magenta"}
                       )
    return fig
    '''
    callback_gen = callback_gen.replace('__MOTOR_ID__', motor_id)
    exec(callback_gen)
    
def setCallbacks():
    for motor_id in ['A', 'B', 'C', 'D', 'E']:
        genCallback(motor_id)

app.layout = setLayouts()
setCallbacks()
    
app.run_server(debug=True, use_reloader=False)

Изменяя ручку target можно сделать много выводов:
- В выборке нет "наложения" точек (нет вырождения)
- Много точки выборки не определены (pgain и vgain принимают значения не из всего диапазона для конкретного motor/screw)
- Почти всегда screw A дает значение target больше чем остальные 
- Для одинакового pgain, vgain данные "скачут" в зависимости от screw, что намекает на нелинейность
- БОльшая часть target-a лежит от 0.13 до 2 (визуально явно больше половины)

## Линейная регрессия

Убедимся экспериментально, что линейная регрессия для данной выборки плохо подходит

In [5]:
import torch
from torch.autograd import Variable
from sklearn.metrics import r2_score

features = ['pgain', 'vgain', 
            'motor_A', 'motor_B', 'motor_C', 'motor_D', 'motor_E',
            'screw_A', 'screw_B', 'screw_C', 'screw_D', 'screw_E']
X = Variable(torch.tensor(df_encoded[features].values))
y = Variable(torch.tensor(df_encoded[target].values))

class TorchLinearRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TorchLinearRegression, self).__init__()
        # perform X * w
        self.linear = torch.nn.Linear(input_dim, output_dim, dtype=torch.float64, bias=False)
        torch.nn.init.xavier_uniform_(self.linear.weight)
 
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y_pred = self.linear(x)
        return y_pred    

torch_model = TorchLinearRegression(len(features), len(target))
 
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(torch_model.parameters(), lr = 0.01)
 
for epoch in range(5000):
    # Forward pass
    pred_y = torch_model(X)
 
    # Compute loss
    loss = loss_fn(pred_y, y)
    # Zero gradients, perform a backward pass, 
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

y_pred = torch_model(X)

print(f'r2_score={r2_score(y.detach().numpy(), y_pred.detach().numpy())}')

r2_score=0.5240160004452085


R2 score низкий, и требуется много итераций для обучения, линейная модель плохо справляется с данным датасетом

## Перцептрон

Создадим класс Perceptron аналогично как на семинаре

In [6]:
class Perceptron(torch.nn.Module):
    @property
    def device(self):
        for p in self.parameters():
            return p.device

    def __init__(self, input_dim, output_dim, num_layers=0, 
                 hidden_dim=64, p=0.0):
        super(Perceptron, self).__init__()
        
        self.layers = torch.nn.Sequential()
        
        prev_size = input_dim
        for i in range(num_layers):
            self.layers.add_module('layer{}'.format(i), 
                                  torch.nn.Linear(prev_size, hidden_dim, dtype=torch.float64))
            self.layers.add_module('relu{}'.format(i), torch.nn.ReLU())
            self.layers.add_module('dropout{}'.format(i), torch.nn.Dropout(p=p))
            prev_size = hidden_dim
        
        self.layers.add_module('classifier', 
                               torch.nn.Linear(prev_size, output_dim, dtype=torch.float64))        
        
    def forward(self, input):
        return self.layers(input)

In [7]:
def testing(model, dataset):
    generator = torch.utils.data.DataLoader(dataset, batch_size=64)

    pred = []
    real = []
    for x, y in generator:
        
        y_pred = model(x)
        pred.extend(y_pred.detach().squeeze().numpy().tolist())
        real.extend(y.detach().numpy().tolist())

    return np.array(real), np.array(pred)

def trainer(model, dataset, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        generator = torch.utils.data.DataLoader(dataset, batch_size=64, 
                                              shuffle=True)
        for x, y in generator:
            optimizer.zero_grad()

            output = model(x)
            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()

Подбор гиперпараметров (num_layers, hidden_dim, p):

In [8]:
# Немного измененный код из семинара 2
# Там была задача классификации, здесь регрессия, поэтому функция потерь это MSE

def draw_table(data, title=['R2_SCORE'], width=[60, 11]):    
    row_format = '|' + '|'.join([("{:>" + str(w) + "}") for w in width]) + '|'
    row_format_bet = '+' + '+'.join([("{:>" + str(w) + "}") for w in width]) + '+'
    
    print(row_format_bet.format(
        "-" * width[0], *["-" * width[i + 1] for i, _ in enumerate(title)]))
    print(row_format.format("", *title))
    print(row_format_bet.format(
        "-" * width[0], *["-" * width[i + 1] for i, _ in enumerate(title)]))
    for key in data:
        if len(key) > width[0]:
            row_name = '...' + key[len(key)-width[0]+3:]
        else:
            row_name = key
        print(row_format.format(row_name, *[round(x, 2) for x in data[key]]))
        print(row_format_bet.format(
            "-" * width[0], *["-" * width[i + 1] for i, _ in enumerate(title)]))

cross_val = KFold(3)
number_of_batch = cross_val.get_n_splits()

grid = ParameterGrid({'num_layers': [2, 4], 
                      'hidden_dim': [8, 64],
                      'p': [0.0, 0.1],
                      'lr': [0.001]})

scores = dict()
for item in tqdm(grid):
    list_of_scores = []
    for train_index, test_index in tqdm(cross_val.split(X), 
                                        total=number_of_batch, leave=False):
        x_train_fold = X[train_index]
        x_test_fold = X[test_index]
        y_train_fold = y[train_index]
        y_test_fold = y[test_index]

        traindata = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
        testdata = torch.utils.data.TensorDataset(x_test_fold, y_test_fold)

        model = Perceptron(input_dim=len(features), output_dim=len(target), num_layers=item['num_layers'], p=item['p'],
                           hidden_dim=item['hidden_dim'])
        _ = model.train()
        trainer(model=model, 
                dataset=traindata, 
                loss_function=torch.nn.MSELoss(), 
                optimizer=torch.optim.Adam(model.parameters(), lr=item['lr']), 
                epochs=1000)
        
        _ = model.eval()
        _, y_pred_fold = testing(model, testdata)
        score = r2_score(y_test_fold, y_pred_fold)
        list_of_scores.append(score)
        
    scores[str(item)] = [np.mean(score_list) for score_list in list_of_scores]
draw_table(scores)


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+------------------------------------------------------------+-----------+
|                                                            |   R2_SCORE|
+------------------------------------------------------------+-----------+
|   {'hidden_dim': 8, 'lr': 0.001, 'num_layers': 2, 'p': 0.0}|       0.74|
+------------------------------------------------------------+-----------+
|   {'hidden_dim': 8, 'lr': 0.001, 'num_layers': 2, 'p': 0.1}|       0.93|
+------------------------------------------------------------+-----------+
|   {'hidden_dim': 8, 'lr': 0.001, 'num_layers': 4, 'p': 0.0}|       0.88|
+------------------------------------------------------------+-----------+
|   {'hidden_dim': 8, 'lr': 0.001, 'num_layers': 4, 'p': 0.1}|       0.93|
+------------------------------------------------------------+-----------+
|  {'hidden_dim': 64, 'lr': 0.001, 'num_layers': 2, 'p': 0.0}|       0.95|
+------------------------------------------------------------+-----------+
|  {'hidden_dim': 64, 'lr

Обучим "сильнее" персептрон с лучшими гиперпараметрами

In [9]:
# Создание dataset-a
X = torch.tensor(df_encoded[features].values)
y = torch.tensor(df_encoded[target].values)
dset = torch.utils.data.TensorDataset(X, y)


# Параметры "лучшего" перцептрона
num_layers=4
hidden_dim=8
p=0.0
lr=0.001

model = Perceptron(input_dim=len(features),output_dim=len(target), num_layers=num_layers, hidden_dim=hidden_dim, p=p)

trainer(model=model,
        dataset=dset,
        loss_function=torch.nn.MSELoss(),
        optimizer=torch.optim.Adam(model.parameters(), lr=lr),
        epochs=5000)

y_pred = model.forward(X)
print(f'r2_score={r2_score(y.detach().numpy(), y_pred.detach().numpy())}')


r2_score=0.989684932087016


Как видно персептрон за такое же количество итераций справляется лучше, чем линейная регрессия

## SVR

Воспользуемся методом опорных векторов для регрессии (Support Vector Regression).

In [10]:
from sklearn.svm import SVR

In [11]:
X = torch.tensor(df_encoded[features].values)
y = torch.tensor(df_encoded[target].values)

In [12]:
cross_val = KFold(3)
number_of_batch = cross_val.get_n_splits()


def varyHyperParams(grid):
    scores = dict()
    for item in tqdm(grid):
        list_of_scores = []
        for train_index, test_index in tqdm(cross_val.split(X), 
                                            total=number_of_batch, leave=False):
            x_train_fold = X[train_index]
            x_test_fold = X[test_index]
            y_train_fold = y[train_index]
            y_test_fold = y[test_index]

            model = SVR(kernel=item['kernel'],
                        C=item['C'],
                        epsilon=item['epsilon'])
            _ = model.fit(x_train_fold, y_train_fold.ravel())

            y_pred_fold = model.predict(x_test_fold)
            score = r2_score(y_test_fold, y_pred_fold)
            list_of_scores.append(score)
        
        scores[str(item)] = [np.mean(score_list) for score_list in list_of_scores]
    draw_table(scores)

grid = ParameterGrid({'kernel': ['linear', 'poly', 'rbf'],
                       'C': [1e-3, 1e-1, 1e1, 1e3],
                       'epsilon': [1e-3, 1e-1]})
varyHyperParams(grid)


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+------------------------------------------------------------+-----------+
|                                                            |   R2_SCORE|
+------------------------------------------------------------+-----------+
|          {'C': 0.001, 'epsilon': 0.001, 'kernel': 'linear'}|      -0.06|
+------------------------------------------------------------+-----------+
|            {'C': 0.001, 'epsilon': 0.001, 'kernel': 'poly'}|      -0.07|
+------------------------------------------------------------+-----------+
|             {'C': 0.001, 'epsilon': 0.001, 'kernel': 'rbf'}|       -0.1|
+------------------------------------------------------------+-----------+
|            {'C': 0.001, 'epsilon': 0.1, 'kernel': 'linear'}|      -0.05|
+------------------------------------------------------------+-----------+
|              {'C': 0.001, 'epsilon': 0.1, 'kernel': 'poly'}|      -0.08|
+------------------------------------------------------------+-----------+
|               {'C': 0.0

Оптимальным кажется выбор параметра kernel: rbf. Попробуем подобрать более подходящие параметры

In [13]:
grid = ParameterGrid({'kernel': ['rbf'],
                      'C': [1e-3, 1e-1, 1e1, 5e2, 1e3],
                      'epsilon': [1e-6, 1e-3, 1e-1, 1e1, 1e3]})

varyHyperParams(grid)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+------------------------------------------------------------+-----------+
|                                                            |   R2_SCORE|
+------------------------------------------------------------+-----------+
|             {'C': 0.001, 'epsilon': 1e-06, 'kernel': 'rbf'}|       -0.1|
+------------------------------------------------------------+-----------+
|             {'C': 0.001, 'epsilon': 0.001, 'kernel': 'rbf'}|       -0.1|
+------------------------------------------------------------+-----------+
|               {'C': 0.001, 'epsilon': 0.1, 'kernel': 'rbf'}|      -0.12|
+------------------------------------------------------------+-----------+
|              {'C': 0.001, 'epsilon': 10.0, 'kernel': 'rbf'}|       -2.2|
+------------------------------------------------------------+-----------+
|            {'C': 0.001, 'epsilon': 1000.0, 'kernel': 'rbf'}|       -2.2|
+------------------------------------------------------------+-----------+
|               {'C': 0.1

Ещё попробуем подобрать параметры C и epsilon

In [15]:
grid = ParameterGrid({'kernel': ['rbf'],
                      'C': [300, 400, 500],
                      'epsilon': [0.05, 0.1, 0.15]})

varyHyperParams(grid)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+------------------------------------------------------------+-----------+
|                                                            |   R2_SCORE|
+------------------------------------------------------------+-----------+
|                {'C': 300, 'epsilon': 0.05, 'kernel': 'rbf'}|        0.9|
+------------------------------------------------------------+-----------+
|                 {'C': 300, 'epsilon': 0.1, 'kernel': 'rbf'}|       0.91|
+------------------------------------------------------------+-----------+
|                {'C': 300, 'epsilon': 0.15, 'kernel': 'rbf'}|        0.9|
+------------------------------------------------------------+-----------+
|                {'C': 400, 'epsilon': 0.05, 'kernel': 'rbf'}|        0.9|
+------------------------------------------------------------+-----------+
|                 {'C': 400, 'epsilon': 0.1, 'kernel': 'rbf'}|       0.91|
+------------------------------------------------------------+-----------+
|                {'C': 40

Уменьшим диапазон гиперпараметров для нахождения наиболее подходящих.

Воспользуемся наилучшим набором гиперпараметров

In [16]:
kernel='rbf'
epsilon=0.1
C=400

model = SVR(kernel=kernel,
            C=C,
            epsilon=epsilon,
            tol=1e-3)
_ = model.fit(X, y.ravel())

y_pred = model.predict(X)
score = r2_score(y, y_pred)
print(score)

0.9310921089493871


Получаем довольно хороший результат

### Анализ результатов
- Как указывалось авторами датасет сильно нелинейный, поэтому линейная регрессия дает плохие результаты
- Наилучшим методом оказался *Perceptron*
### Возникшие проблемы
- Датасет тяжело визуализировать, возможно пять 3-х мерные графиков это не лучшая идея для такого датасета